In [ ]:
import numpy as np
import h5py
from numpy.fft import fftshift
from scipy.ndimage.measurements import center_of_mass
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import glob

# This imports all necessary operators. GPU will be auto-selected
from pynx.cdi import *
from pynx.utils.math import smaller_primes

# Which matplotlib to use
* %inline, plots do not update
* %notebook on jupyter notebook
* %ipyml on jupyter lab, not supported yet

In [ ]:
%matplotlib notebook

# Make sure you are in the right folder/using the right data

In [ ]:
glob.glob("*.npz")

# Extract data

In [ ]:
nrj = 12.996
wavelength = 12.384 / nrj * 1e-10
print("  CXI input: Energy = %8.2fkeV" % nrj)
print(f"  CXI input: Wavelength = {wavelength*1e10} A")

detector_distance = 0.83
print("  CXI input: detector distance = %8.2fm" % detector_distance)

pixel_size_detector = 55e-6
print("  CXI input: detector pixel size = %8.2fum" % (pixel_size_detector * 1e6))

scan = glob.glob("*_pynx_align*.npz")[0].split("_")[0]
print("  Scan n°", scan)

iobs = np.load(glob.glob("*_pynx_align*.npz")[0])["data"]
print("  CXI input: loading data")

mask = np.load(glob.glob("*maskpynx*.npz")[0])["mask"].astype(np.int8)
nb = mask.sum()
print("  CXI input: loading mask, with %d pixels masked (%6.3f%%)" % (nb, nb * 100 / mask.size))

# Centre & crop data

Crop data around center of mass, with a maximum size along the 3 directions

In [ ]:
max_size = 256
if iobs.ndim == 3:
    nz0, ny0, nx0 = iobs.shape
    
    # Find center of mass
    z0, y0, x0 = center_of_mass(iobs)
    print("Center of mass at:", z0, y0, x0)
    iz0, iy0, ix0 = int(round(z0)), int(round(y0)), int(round(x0))
    
    # Max symmetrical box around center of mass
    nx = 2 * min(ix0, nx0 - ix0)
    ny = 2 * min(iy0, ny0 - iy0)
    nz = 2 * min(iz0, nz0 - iz0)
    
    if max_size is not None:
        nx = min(nx, max_size)
        ny = min(ny, max_size)
        nz = min(nz, max_size)
        
    # Crop data to fulfill FFT size requirements
    nz1, ny1, nx1 = smaller_primes((nz, ny, nx), maxprime=7, required_dividers=(2,))

    print("Centering & reshaping data: (%d, %d, %d) -> (%d, %d, %d)" % (nz0, ny0, nx0, nz1, ny1, nx1))
    iobs = iobs[iz0 - nz1 // 2:iz0 + nz1 // 2, iy0 - ny1 // 2:iy0 + ny1 // 2,
                ix0 - nx1 // 2:ix0 + nx1 // 2]
    if mask is not None:
        mask = mask[iz0 - nz1 // 2:iz0 + nz1 // 2, iy0 - ny1 // 2:iy0 + ny1 // 2,
                    ix0 - nx1 // 2:ix0 + nx1 // 2]
        print("Centering & reshaping mask: (%d, %d, %d) -> (%d, %d, %d)" % (nz0, ny0, nx0, nz1, ny1, nx1))
        
else:
    ny0, nx0 = iobs.shape
    
    # Find center of mass
    y0, x0 = center_of_mass(iobs)
    print("Center of mass at:", y0, x0)
    iy0, ix0 = int(round(y0)), int(round(x0))
    
    # Max symmetrical box around center of mass
    nx = 2 * min(ix0, nx0 - ix0)
    ny = 2 * min(iy0, ny0 - iy0)
    if max_size is not None:
        nx = min(nx, max_size)
        ny = min(ny, max_size)
        nz = min(nz, max_size)
        
    # Crop data to fulfill FFT size requirements
    ny1, nx1 = smaller_primes((ny, nx), maxprime=7, required_dividers=(2,))

    print("Centering & reshaping data: (%d, %d) -> (%d, %d)" % (ny0, nx0, ny1, nx1))
    iobs = iobs[iy0 - ny1 // 2:iy0 + ny1 // 2, ix0 - nx1 // 2:ix0 + nx1 // 2]
    
    if mask is not None:
        mask = mask[iy0 - ny1 // 2:iy0 + ny1 // 2, ix0 - nx1 // 2:ix0 + nx1 // 2]

# Run 1

To define the support if not good enough, can be sufficient otherwise

In [ ]:
nrun = 5

for i in range(nrun):
    print(f"Run {i}")
    
    # Create cdi object with data and mask, laod the main parameters
    cdi = CDI(fftshift(iobs),
              support = None,
              obj = None,
              mask = fftshift(mask),
              wavelength = wavelength,
              pixel_size_detector = pixel_size_detector,
             detector_distance = detector_distance)
    
    if i==0:
        cdi.save_data_cxi(
            filename = "DiffractionData.cxi",
            sample_name = "III-B20",
            experiment_id = "HC4050",
            instrument = "ID01")

    # Change support threshold for supports update
    threshold_relative = np.random.uniform(0.25, 0.33)
    print(f"Threshold: {threshold_relative}")
    
    sup = SupportUpdate(
        threshold_relative = threshold_relative,
        smooth_width=(2, 1, 600),
        force_shrink = False,
        method='rms', 
        post_expand = (1, -2, 1),
    )
    
    # Initialize the free pixels for LLK
    cdi = InitFreePixels() * cdi

    # Initialize the support with autocorrelation
    cdi = ShowCDI() * ScaleObj() * AutoCorrelationSupport(
        threshold = 0.1,
        verbose = True) * cdi

    # Begin with HIO cycles without PSF and with support updates
    try:
        cdi = (sup * HIO(beta=0.9, calc_llk=50, show_cdi=50)**50)**8 * cdi
        cdi = (sup * RAAR(beta=0.9, calc_llk=50, show_cdi=50)**50)**10 * cdi

        # PSF is introduced at 66% of HIO and RAAR so from cycle n°924
        cdi = InitPSF(
            model = "gaussian",
            fwhm = 0.3,
            # eta = 0.1,
        ) * cdi

        cdi = (sup * RAAR(beta=0.9, calc_llk=50, show_cdi=50, update_psf=20)**50)**10 * cdi
        cdi = (sup * ER(calc_llk=50, show_cdi=50, update_psf=20)**50)**6 * cdi

        cdi.save_obj_cxi("reconstructions/result_scan_{}_run_{}_LLK_{:.4}_support_{:.4}_autocorrelation.cxi".format(scan,
                                                                                         i,
                                                                                         cdi.get_llk()[0],
                                                                                        threshold_relative))
    
    except SupportTooLarge:
        print("Threshold value probably too low, support too large too continue")
        pass
        
    
    print("\n##########################################################################################################\n")

In [ ]:
!/home/esrf/favre/dev/devel.p9/bin/pynx-cdi-analysis.py *LLK* modes mode_crop=no modes_output=modes.h5

# Create support

In [ ]:
%matplotlib inline

In [ ]:
good_scan = "result_run1_no_sup_psf.cxi"

In [ ]:
# Get data

with h5py.File(good_scan, "r") as f:
    data = f["entry_1"]["data_1"]["data"][:]

plt.imshow(np.abs(data[data.shape[0]//2, :, :][:]))

In [ ]:
# Create support 

amp = np.abs(data)
threshold = 0.05
support = np.where(amp > threshold * np.max(amp), 1, 0)

plt.imshow(support[data.shape[0]//2, :, :][:])

In [ ]:
sigma = 0.8

bigdata = 100 * support
conv_support = np.where(gaussian_filter(bigdata, sigma) != 0, 1, 0)

plt.imshow(conv_support[data.shape[0]//2, :, :][:])

np.savez(f"support_filter_sig_{sigma}_threshold_{threshold}.npz", oldmask = support, mask = conv_support)

# Get support

In [ ]:
support = np.load(f"support_filter_sig_{sigma}_threshold_{threshold}.npz")["mask"].astype("int8")

# Or you can simply get the support from the last scan
# Another solution is to reload the saved file and to get its support
# cdi.get_support()

# Run 2

In [ ]:
nrun = 5

for i in range(nrun):
    print(f"Run {i}")
    
    # Create cdi object with data and mask, laod the main parameters
    cdi = CDI(fftshift(iobs),
              support = support,
              obj = None,
              mask = fftshift(mask),
              wavelength = wavelength,
              pixel_size_detector = pixel_size_detector,
             detector_distance = detector_distance)
    
    if i==0:
        cdi.save_data_cxi(
            filename = "DiffractionData.cxi",
            sample_name = "III-B20",
            experiment_id = "HC4050",
            instrument = "ID01")

    # Change support threshold for supports update
    threshold_relative = np.random.uniform(0.25, 0.33)
    print(f"Threshold: {threshold_relative}")
    
    sup = SupportUpdate(
        threshold_relative = threshold_relative,
        smooth_width=(2, 1, 600),
        force_shrink = False,
        method='rms', 
        post_expand = (1, -2, 1),
    )
    
    # Initialize the free pixels for LLK
    cdi = InitFreePixels() * cdi

    # Initialize the support with autocorrelation
    cdi = ShowCDI() * ScaleObj() * AutoCorrelationSupport(
        threshold = 0.1,
        verbose = True) * cdi

    # Begin with HIO cycles without PSF and with support updates
    try:
        cdi = (sup * HIO(beta=0.9, calc_llk=50, show_cdi=50)**50)**8 * cdi
        cdi = (sup * RAAR(beta=0.9, calc_llk=50, show_cdi=50)**50)**10 * cdi

        # PSF is introduced at 66% of HIO and RAAR so from cycle n°924
        cdi = InitPSF(
            model = "gaussian",
            fwhm = 0.3,
            # eta = 0.1,
        ) * cdi

        cdi = (sup * RAAR(beta=0.9, calc_llk=50, show_cdi=50, update_psf=20)**50)**10 * cdi
        cdi = (sup * ER(calc_llk=50, show_cdi=50, update_psf=20)**50)**6 * cdi

        cdi.save_obj_cxi("result_scan_{}_run_{}_LLK_{:.4}_support_{:.4}_autocorrelation.cxi".format(scan,
                                                                                         i,
                                                                                         cdi.get_llk()[0],
                                                                                        threshold_relative))
    
    except SupportTooLarge:
        print("Threshold value probably too low, support too large too continue")
        pass
        
    
    print("\n##########################################################################################################\n")